In [4]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [7]:
import anvil.server
import pickle
from sklearn.preprocessing import LabelEncoder

anvil.server.connect("QZKAAXCCOF4RLGUOJ6OTQEMY-ZM4CRXXYLGDPQ6LK")

In [8]:
from ipywidgets.widgets.trait_types import date_from_json
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import anvil.tables as tables
from anvil.tables import app_tables
import anvil.media

@anvil.server.callable
def import_csv_data(file):
  with anvil.media.TempFile(file) as f:
    df = pd.read_csv(f)

    df.drop(['Description'], axis=1, inplace=True)
    
    col = []
    for i in zip(df.columns.values,df.dtypes):
        if i[1] == 'O':
            col.append(i[0])
    df[col]=df[col].astype('category')

    labelencoder = LabelEncoder()
    for j,i in df.iteritems():
      if i.dtype == 'category'or i.dtype == 'bool':
        df[j] = labelencoder.fit_transform(df[j])

    df['Start_Time'] = pd.to_datetime(df['Start_Time'])
    df['End_Time'] = pd.to_datetime(df['End_Time'])
    df['Duration'] = df.End_Time - df.Start_Time 
    df['Duration'] = df['Duration'].apply(lambda x:round(x.total_seconds() / 60) )
    df['Year'] = df['Start_Time'].dt.year

    df['Month'] = df['Start_Time'].dt.month
    df['Weekday']= df['Start_Time'].dt.weekday
    df.drop(['Start_Time','Number','End_Time','Weather_Timestamp','Start_Lat','Start_Lng','Country','Airport_Code'], axis=1, inplace=True)

    scaler = MinMaxScaler()
    features = ['Temperature(F)','Distance(mi)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)','Duration','Year', 'Month','Weekday']
    df[features] = scaler.fit_transform(df[features])

    loaded_model = pickle.load(open('final_model.sav', 'rb'))
    score = loaded_model.predict(df)
    return (score)